In [2]:
import dask
from dask.distributed import Client
import dask.dataframe as dd
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from scipy import sparse
import psutil
NUM_WORKER = psutil.cpu_count(logical = False)

In [4]:
df = dd.read_csv('../data/datasets/interim/features/*.csv')

In [8]:
df.head()

,block,invocation,package,method_name,app,malware
0,public constructor <init>()Va.smali,invoke-direct,Landroid/arch/a/a/b;,<init>,%D8%AA%D9%84%D8%A7%D9%88%D8%A7%D8%AA-%D8%AE%D8...,0
1,public constructor <init>()Va.smali,invoke-direct,Ljava/util/HashMap;,<init>,%D8%AA%D9%84%D8%A7%D9%88%D8%A7%D8%AA-%D8%AE%D8...,0
2,public a(Ljava/lang/Object;)Za.smali,invoke-virtual,Ljava/util/HashMap;,containsKey,%D8%AA%D9%84%D8%A7%D9%88%D8%A7%D8%AA-%D8%AE%D8...,0
3,constructor <init>(Landroid/arch/a/a/b$c;Landr...,invoke-direct,Landroid/arch/a/a/b$e;,<init>,%D8%AA%D9%84%D8%A7%D9%88%D8%A7%D8%AA-%D8%AE%D8...,0
4,constructor <init>(Landroid/arch/a/a/b$c;Landr...,invoke-direct,Landroid/arch/a/a/b$e;,<init>,%D8%AA%D9%84%D8%A7%D9%88%D8%A7%D8%AA-%D8%AE%D8...,0
